In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import igraph as ig
from tqdm import tqdm_notebook as tqdm

### Config

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc('ps',fonttype = 42)
plt.rc('pdf',fonttype = 42)
plt.rcParams.update({'font.size': 20})
plt.rcParams['ps.useafm'] = True
plt.rcParams['pdf.use14corefonts'] = True
plt.rcParams['axes.unicode_minus'] = False

### Load data

In [ ]:
texts_df = pd.read_pickle("data/processed/texts.p")

### Prepare data

#### Fix authors

In [ ]:
author_fix = {
    "Bachiller D. P. Gatell": "Bachiller D. P. Gatell.",
    "Eliza Haywood": "Eliza Fowler Haywood",
}
texts_df["author"] = texts_df["author"].replace(author_fix)

In [ ]:
topics_by_author_df = texts_df.groupby("author")["topics"].agg(lambda x: set.union(*x))

### Create Graph

In [ ]:
g = ig.Graph(directed=False)

In [ ]:
for a in texts_df["author"].unique():
    #if "Anonym" in a or "Anonyme" in a or "Anónimo" in a or "[" in a or "missing" in a:
    #    continue
    g.add_vertex(a, type=True)

In [ ]:
for t in set.union(*texts_df.groupby(["filename", "author", "language"])["topics"].agg(lambda x: set.intersection(*x))):
    g.add_vertex(t, type=False)

In [ ]:
list(set.union(*texts_df.groupby(["filename", "author", "language"])["topics"].agg(lambda x: set.intersection(*x))))

In [ ]:
for a, t in topics_by_author_df.items():
    for cur_t in t:
        if a in g.vs["name"]:
            g.add_edge(a, cur_t)

In [ ]:
color_dict = {False: "green", True: "red"}
visual_style = {}
visual_style["vertex_size"] = g.degree()
visual_style["vertex_label"] = g.vs["name"]
visual_style["vertex_label_size"] = 8
visual_style["vertex_color"] = [color_dict[i] for i in g.vs["type"]]
visual_style["edge_color"] = "grey"
#visual_style["layout"] = g.layout_bipartite(g.vs["type"], hgap=1000*1000)
visual_style["layout"] = g.layout_kamada_kawai()
visual_style["bbox"] = (1000, 1000)
visual_style["margin"] = 100
ig.plot(g, "author_topics_graph.pdf", **visual_style)

In [ ]:
g_topics, g_authors = g.bipartite_projection()

In [ ]:
visual_style = {}
visual_style["vertex_size"] = 50
visual_style["vertex_label"] = g_topics.vs["name"]
visual_style["vertex_label_size"] = 8
visual_style["layout"] = g_topics.layout_kamada_kawai()
visual_style["bbox"] = (1000, 1000)
visual_style["margin"] = 50
ig.plot(g_topics, **visual_style)

In [ ]:
visual_style = {}
visual_style["vertex_size"] = 50
visual_style["vertex_label"] = g_authors.vs["name"]
visual_style["vertex_label_size"] = 8
visual_style["layout"] = g_authors.layout_kamada_kawai()
visual_style["bbox"] = (1000, 1000)
visual_style["margin"] = 50
ig.plot(g_authors, **visual_style)